In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
test_data = pd.read_csv('../input/covid19-global-forecasting-week-1/test.csv',index_col=0)
train_data = pd.read_csv('../input/covid19-global-forecasting-week-1/train.csv',index_col=0)
sub_sample = pd.read_csv('../input/covid19-global-forecasting-week-1/submission.csv')

#test_data.head()
#train_data.head()
#sub_sample.head()

In [ ]:
test_data=test_data.rename(columns={'Country/Region':'Country','Province/State':'Province'})
train_data=train_data.rename(columns={'Country/Region':'Country','Province/State':'Province'})

In [ ]:
#train_data.isnull().sum()
#test_data.isnull().sum()

In [ ]:
#some countries don't have provinces
#we'll create a new column that sums the country and province name and remove the province column
train_data.fillna(1)
train_data['Province_new'] = train_data['Country']+'_'+train_data['Province'].astype(str)
#train_data.head()
test_data['Province_new'] = test_data['Country']+'_'+test_data['Province'].astype(str)
#test_data.head()

In [ ]:
#test_data.groupby('Date').Date.count()
#test_data.groupby('Country').Country.count()
#test_data.Province.value_counts()
#test_data.Province.describe()
#test_data.Country.value_counts()
#train_data.groupby('Date').Date.count()

In [ ]:
#test_data.Date.agg([min,max])
#train_data.Date.agg([min,max])

In [ ]:
# Convert categorical data in Date to actual time dates
from datetime import date
test_data['Date'] = pd.to_datetime(test_data['Date'])
train_data['Date'] = pd.to_datetime(train_data['Date'])

In [ ]:
#create day,month,year columns
train_data['Day'] = train_data['Date'].dt.day
train_data['Month'] = train_data['Date'].dt.month
train_data['Year'] = train_data['Date'].dt.year

test_data['Day'] = test_data['Date'].dt.day
test_data['Month'] = test_data['Date'].dt.month
test_data['Year'] = test_data['Date'].dt.year

#create column with how many days from the start of the measuring period
first_day = train_data.Date.min()
train_data['Days_since_start'] = (train_data['Date']-first_day).dt.days
test_data['Days_since_start'] = (test_data['Date']-first_day).dt.days

In [ ]:
#select only the training data before 11/03/2020 so the train and test data don't intersect
train_data_reduced = train_data.drop(train_data[train_data.Date>'2020-3-11'].index)

In [ ]:
#We're going to remove the original 'Province' column 
#and the Date column since it's not numerical and we have extracted the info in other numerical columns
train_data_reduced = train_data_reduced.drop(['Province','Date'],axis=1)
test_data = test_data.drop(['Province','Date'],axis=1)

train_data_reduced['Lat'] = train_data_reduced['Lat'].astype(int)
train_data_reduced['Long'] = train_data_reduced['Long'].astype(int)
test_data['Lat'] = test_data['Lat'].astype(int)
test_data['Long'] = test_data['Long'].astype(int)

In [ ]:
# deal with the cathegorical data
#from sklearn.preprocessing import LabelEncoder
#cat_features = ['Country','Province_new']
#encoder = LabelEncoder()
#train_data_reduced[cat_features] = train_data_reduced[cat_features].apply(encoder.fit_transform)
#test_data[cat_features] = test_data[cat_features].apply(encoder.transform)
#data_cols_train = train_data.drop(cat_features)


In [ ]:
#list(encoder.classes_)

In [ ]:
#print(train_data_reduced.Country.nunique())
#print(train_data_reduced.Province_new.nunique())
#print(test_data.Country.nunique())
#print(test_data.Province_new.nunique())

In [ ]:
#print(train_data_reduced.Country.unique())
#print(test_data.Country.unique())
#they have the same values for Country and Province_new

In [ ]:
# create dictionary for countries and provinces
country_labels = train_data_reduced['Country'].astype('category').cat.categories.tolist()
country_dict = {'Country':{k:v for k,v in zip(country_labels,list(range(1,len(country_labels)+1)))}}
train_data_reduced.replace(country_dict, inplace=True)

province_labels = train_data_reduced['Province_new'].astype('category').cat.categories.tolist()
province_dict = {'Province_new':{k:v for k,v in zip(province_labels,list(range(1,len(province_labels)+1)))}}
train_data_reduced.replace(province_dict, inplace=True)

test_data.replace(country_dict,inplace=True)
test_data.replace(province_dict, inplace=True)


In [ ]:
#separate target from data
#create validation set from train data
#from sklearn.model_selection import train_test_split
X_ = train_data_reduced.drop(['ConfirmedCases','Fatalities'],axis=1)
y_cases = train_data_reduced['ConfirmedCases']
y_deaths = train_data_reduced['Fatalities']

#X_train, X_valid, y_cases_train, y_cases_valid,y_deaths_train, y_deaths_valid = train_test_split(
#X_,y_cases,y_deaths,test_size=0.2, random_state=1)  

#apply a model
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
xgbreg = xgb.XGBRegressor()

parameters = {'objective':['reg:linear'],
              'learning_rate': [.05, 0.15, .3], 
              'max_depth': [5, 6, 7],
              'min_child_weight': [1],
              'silent': [1],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [100, 250, 500]}

model_cases = GridSearchCV(xgbreg,
                        parameters,
                        cv = 2,
                        n_jobs = 5,
                        verbose=True)

model_deaths = GridSearchCV(xgbreg,
                        parameters,
                        cv = 2,
                        n_jobs = 5,
                        verbose=True)
#model_cases.fit(X_train,y_cases_train)
#model_deaths.fit(X_train,y_deaths_train)

#y_cases_pred = model_cases.predict(X_valid)
#y_deaths_pred = model_deaths.predict(X_valid)

model_cases.fit(X_,y_cases)
model_deaths.fit(X_,y_deaths)


from sklearn.metrics import accuracy_score
accuracy_cases = accuracy_score(y_cases_valid,y_cases_pred)
accuracy_deaths = accuracy_score(y_deaths_valid,y_deaths_pred)
print(accuracy_cases,accuracy_deaths)

In [ ]:
#make predictions on test data
cases =model_cases.predict(test_data).round()
deaths = model_deaths.predict(test_data).round()
output = pd.DataFrame({'ForecastId': test_data.index,
                       'ConfirmedCases': cases,
                       'Fatalities':deaths})
output.to_csv('submission.csv', index=False)